In [ ]:
from peak_performance import pipeline as pl
from peak_performance import models
from peak_performance import plots

In [1]:
import models
import plots
import pipeline as pl
import pandas
import numpy as np
import arviz as az

User information

In [4]:
# 1: Specify the absolute path to the raw data files.
path_raw_data = r"C:\Users\niesser\Desktop\Local GitLab Repositories\peak-performance\peak_performance\example"

# obtain a list of raw data file names.
raw_data_files = pl.detect_npy(path_raw_data)
# raw_data_files = ["A1t1R1Part2_1_110_109.9_110.1.npy", "A1t1R1Part2_2_111_109.9_110.1.npy"]
# here, the names still contain the data type suffix which is removed by the following line.
# raw_data_files = [file[:-4] for file in raw_data_files]
print(raw_data_files)

# necessary information from the user
double_peak = 5*[False]           # 2: List with Booleans in the same order as raw_data_files. Set to True for a given signal, if the signal contains a double peak, and set to False, if it contains a single peak. Visually check this beforehand.
pre_filtering = True                # 3: Set this variable to True if you want to check for peaks before fitting/sampling to potentially save a lot of computation time. If you choose True, then you have to provide an expected retention time for each signal.
retention_time_estimate = 5*[26.2]    # 4: in case you set pre_filtering to True, give a retention time estimate (float) for each signal in raw_data_files. In case of a double peak, give two retention times (in chronological order) as a tuple containing two floats.
peak_width_estimate = 1             # 5: in case you set pre_filtering to True, give a rough estimate of the average peak width in minutes you would expect for your LC-MS method.
minimum_sn = 5                      # 6: in case you set pre_filtering to True, give a minimum signal to noise ratio for a signal to be defined as a peak during pre-filtering

['A1t1R1Part2_1_110_109.9_110.1.npy', 'A1t1R1Part2_2_111_109.9_110.1.npy', 'A1t1R1Part2_3_111_110.9_111.1.npy', 'A1t1R1Part2_4_112_110.9_111.1.npy', 'A1t1R1Part2_5_112_111.9_112.1.npy']


Pipeline

In [5]:
# create data structure and DataFrame(s) for results 
df_summary, path_results = pl.initiate(path_raw_data)
for file in raw_data_files:
    print(f"{file}")
    # parse the data and extract information from the (standardized) file name
    timeseries, acquisition, experiment, precursor_mz, product_mz_start, product_mz_end = pl.parse_data(path_raw_data, file)
    # instantiate the UserInput class all given information
    ui = pl.UserInput(path_results, raw_data_files, double_peak, retention_time_estimate, peak_width_estimate, pre_filtering, minimum_sn, timeseries, acquisition, experiment, precursor_mz, product_mz_start, product_mz_end)
    # calculate initial guesses for pre-filtering and defining prior probability distributions
    slope_guess, intercept_guess, noise_guess = models.initial_guesses(ui.timeseries[0], ui.timeseries[1])
    # apply pre-sampling filter (if selected)
    if pre_filtering:
        prefilter, df_summary = pl.prefiltering(file, ui, noise_guess, df_summary)
        if not prefilter:
            # if no peak candidates were found, continue with the next signal
            plots.plot_raw_data(file, ui)
            continue
        print(f"{ui.experiment} survived prefiltering.")
    # model selection
    if ui.user_info[file][0]:
        # double peak model
        pmodel = models.define_model_doublepeak(ui.timeseries[0], ui.timeseries[1])
    else:
        # single peaks are first modeled with a skew normal distribution
        pmodel = models.define_model_skew(ui.timeseries[0], ui.timeseries[1])
    # sample the chosen model
    print("pre_sampling")
    idata = pl.sampling(pmodel)
    print("post-sampling")
    # apply post-sampling filter
    resample, discard, df_summary = pl.postfiltering(file, idata, ui, df_summary)
    print(f"{resample}, {discard}")
    # if peak was discarded, continue with the next signal
    if discard:
        plots.plot_posterior(file, ui, idata, True)
        print("discarded")
        continue
    # if convergence was not yet reached, sample again with more tuning samples
    if resample:
        print("start resampling")
        idata = pl.sampling(pmodel, tune = 4000)
        resample, discard, df_summary = pl.postfiltering(file, idata, ui, df_summary)
        if discard:
            plots.plot_posterior(f"{file}", ui, idata, True)
            continue
        if resample:
            # if signal was flagged for re-sampling a second time, discard it
            # TODO: should this really be discarded or should the contents of idata be added with an additional comment? (would need to add a comment column)
            df_summary = pl.report_add_nan_to_summary(file, ui, df_summary)
            plots.plot_posterior(f"{file}", ui, idata, True)
            continue
    print("after resampling")
    # add inference data to df_summary and save it as an Excel file
    df_summary = pl.report_add_data_to_summary(file, idata, df_summary, ui)
    # perform posterior predictive sampling
    idata = pl.posterior_predictive_sampling(pmodel, idata)
    # save the inference data object in a zip file
    pl.report_save_idata(idata, ui, file)
    # plot data
    plots.plot_posterior_predictive(file, ui, idata, False)
    plots.plot_posterior(file, ui, idata, False)
# save condesed Excel file with area data
pl.report_area_sheet(path_results, df_summary)

A1t1R1Part2_1_110_109.9_110.1.npy
1 survived prefiltering.
pre_sampling


Sampling: [L, alpha, area, baseline_intercept, baseline_slope, mean, noise, std]
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [baseline_intercept, baseline_slope, noise, mean, std, alpha, area]


Sampling 4 chains for 2_000 tune and 2_000 draw iterations (8_000 + 8_000 draws total) took 103 seconds.


post-sampling
False, False
after resampling


Sampling: [L]


...

Sampling 4 chains for 2_000 tune and 2_000 draw iterations (8_000 + 8_000 draws total) took 126 seconds.


post-sampling
False, True
discarded
A1t1R1Part2_3_111_110.9_111.1.npy
A1t1R1Part2_4_112_110.9_111.1.npy
4 survived prefiltering.
pre_sampling


c:\Users\niesser\Desktop\Local GitLab Repositories\peak-performance\peak_performance\pipeline.py:392: RuntimeWarning: divide by zero encountered in scalar divide
  and ui.timeseries[1][peak] / noise_width_guess > ui.minimum_sn
c:\Users\niesser\Desktop\Local GitLab Repositories\peak-performance\peak_performance\pipeline.py:394: RuntimeWarning: invalid value encountered in scalar divide
  and ui.timeseries[1][peak - 1] / noise_width_guess > 2
c:\Users\niesser\Desktop\Local GitLab Repositories\peak-performance\peak_performance\pipeline.py:394: RuntimeWarning: divide by zero encountered in scalar divide
  and ui.timeseries[1][peak - 1] / noise_width_guess > 2
c:\Users\niesser\Desktop\Local GitLab Repositories\peak-performance\peak_performance\pipeline.py:395: RuntimeWarning: divide by zero encountered in scalar divide
  and ui.timeseries[1][peak + 1] / noise_width_guess > 2
Sampling: [L, alpha, area, baseline_intercept, baseline_slope, mean, noise, std]
Auto-assigning NUTS sampler...
Initi

Sampling 4 chains for 2_000 tune and 2_000 draw iterations (8_000 + 8_000 draws total) took 141 seconds.


post-sampling
False, True
discarded
A1t1R1Part2_5_112_111.9_112.1.npy


In [8]:
df_summary

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat,acquisition,experiment,precursor_mz,product_mz_start,product_mz_end,double_peak
baseline_intercept,5.591,2.835,0.18,10.858,0.035,0.026,6637.0,4777.0,1.0,A1t1R1Part2,1,110.0,109.9,110.1,
baseline_slope,5.032,0.516,4.037,5.962,0.007,0.005,4833.0,5115.0,1.0,A1t1R1Part2,1,110.0,109.9,110.1,
mean,25.951,0.013,25.927,25.975,0.0,0.0,3681.0,3472.0,1.0,A1t1R1Part2,1,110.0,109.9,110.1,
noise,104.272,7.667,89.684,118.588,0.091,0.065,6977.0,4620.0,1.0,A1t1R1Part2,1,110.0,109.9,110.1,
std,0.522,0.022,0.48,0.562,0.0,0.0,3416.0,3232.0,1.0,A1t1R1Part2,1,110.0,109.9,110.1,
area,1500.836,38.702,1430.199,1575.967,0.576,0.408,4514.0,5089.0,1.0,A1t1R1Part2,1,110.0,109.9,110.1,
height,1871.266,38.439,1799.032,1943.768,0.427,0.302,8120.0,6949.0,1.0,A1t1R1Part2,1,110.0,109.9,110.1,
sn,18.043,1.37,15.481,20.596,0.017,0.012,6656.0,4771.0,1.0,A1t1R1Part2,1,110.0,109.9,110.1,
baseline_intercept,[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],A1t1R1Part2,2,111.0,109.9,110.1,False
baseline_slope,[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],A1t1R1Part2,2,111.0,109.9,110.1,False


In [7]:
if any(list(az.summary(idata).loc[:, "r_hat"])) > 1.05:
    print("y")

In [17]:
def postfiltering(filename, idata, ui, df_summary):
    """
    Method to filter out false positive peaks after sampling based on the obtained uncertainties of several peak parameters.

    Parameters
    ----------
    filename
        Name of the raw data file.
    idata
        Inference data object resulting from sampling.
    ui
        Instance of the UserInput class.

    Returns
    -------
    acceptance
        True if the signal was accepted as a peak -> save data and continue with next signal.
        False if the signal was not accepted as a peak -> re-sampling with more tuning samples or discard signal.
    resample
        True: re-sample with more tuning samples, False: don't.
    discard
        True: discard sample.
    """
    # check whether convergence, i.e. r_hat <= 1.05, was not reached OR peak criteria were not met
    doublepeak = ui.user_info[filename][0]
    resample = False
    discard = False
    if not doublepeak == True:
        # for single peak
        if (
            any(list(az.summary(idata).loc[:, "r_hat"])) > 0.99
            or az.summary(idata).loc["std", :]["mean"] <= 0.1
            or az.summary(idata).loc["area", :]["sd"]
            > az.summary(idata).loc["area", :]["mean"] * 0.2
            or az.summary(idata).loc["height", :]["sd"]
            > az.summary(idata).loc["height", :]["mean"] * 0.2
        ):
            # decide whether to discard signal or sample with more tune samples based on size of sigma parameter of normal distribution (std) and on the relative sizes of standard deviations of area and height
            if (
                az.summary(idata).loc["std", :]["mean"] <= 0.1
                or az.summary(idata).loc["area", :]["sd"]
                > az.summary(idata).loc["area", :]["mean"] * 0.2
                or az.summary(idata).loc["height", :]["sd"]
                > az.summary(idata).loc["height", :]["mean"] * 0.2
            ):
                # post-fit check failed
                # add NaN values to summary DataFrame
                # df_summary = report_add_nan_to_summary(filename, ui, df_summary)
                resample = False
                discard = True
            else:
                # r_hat failed but rest of post-fit check passed
                # sample again with more tune samples to possibly reach convergence yet
                resample = True
                discard = False
    return resample, discard, df_summary

In [18]:
resample, discard, df_summary = postfiltering(file, idata, ui, df_summary)

In [19]:
print(resample, discard)

True False


In [13]:
df_summary

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat,acquisition,experiment,precursor_mz,product_mz_start,product_mz_end,double_peak


In [4]:
df_summary

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat,acquisition,experiment,precursor_mz,product_mz_start,product_mz_end,double_peak
baseline_intercept,[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],A1t1R1Part2,3,111.0,110.9,111.1,False
baseline_slope,[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],A1t1R1Part2,3,111.0,110.9,111.1,False
mean,[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],A1t1R1Part2,3,111.0,110.9,111.1,False
noise,[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],A1t1R1Part2,3,111.0,110.9,111.1,False
std,[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],A1t1R1Part2,3,111.0,110.9,111.1,False
area,[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],A1t1R1Part2,3,111.0,110.9,111.1,False
height,[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],A1t1R1Part2,3,111.0,110.9,111.1,False
sn,[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],A1t1R1Part2,3,111.0,110.9,111.1,False
baseline_intercept,[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],A1t1R1Part2,5,112.0,111.9,112.1,False
baseline_slope,[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],A1t1R1Part2,5,112.0,111.9,112.1,False


In [6]:
parameters = [
    "baseline_intercept",
    "baseline_slope",
    "mean",
    "noise",
    "std",
    "area",
    "height",
    "sn",
]
df = pandas.DataFrame(columns=parameters)
df["baseline_intercept"] = len(parameters) * [f"acq"]
df["baseline_slope"] = len(parameters) * [f"acq"]
df["mean"] = len(parameters) * [f"acq"]
df["noise"] = len(parameters) * [f"acq"]
df["std"] = len(parameters) * [f"acq"]
df["area"] = len(parameters) * [f"acq"]
df["height"] = len(parameters) * [f"acq"]
df["sn"] = len(parameters) * [f"acq"]
df["acquisition"] = len(parameters) * [f"acq"]
# TODO: auf type achten, nicht überall str
df["experiment"] = len(parameters) * [f"exp"]
df["precursor_mz"] = len(parameters) * [f"premz"]
df["product_mz_start"] = len(parameters) * [f"pro_mz_strat"]
df["product_mz_end"] = len(parameters) * [f"promzend"]
df["double_peak"] = len(parameters) * [""]
df

,baseline_intercept,baseline_slope,mean,noise,std,area,height,sn,acquisition,experiment,precursor_mz,product_mz_start,product_mz_end,double_peak
0,acq,acq,acq,acq,acq,acq,acq,acq,acq,exp,premz,pro_mz_strat,promzend,
1,acq,acq,acq,acq,acq,acq,acq,acq,acq,exp,premz,pro_mz_strat,promzend,
2,acq,acq,acq,acq,acq,acq,acq,acq,acq,exp,premz,pro_mz_strat,promzend,
3,acq,acq,acq,acq,acq,acq,acq,acq,acq,exp,premz,pro_mz_strat,promzend,
4,acq,acq,acq,acq,acq,acq,acq,acq,acq,exp,premz,pro_mz_strat,promzend,
5,acq,acq,acq,acq,acq,acq,acq,acq,acq,exp,premz,pro_mz_strat,promzend,
6,acq,acq,acq,acq,acq,acq,acq,acq,acq,exp,premz,pro_mz_strat,promzend,
7,acq,acq,acq,acq,acq,acq,acq,acq,acq,exp,premz,pro_mz_strat,promzend,


In [7]:
df_summary = pandas.concat([df_summary, df])

In [8]:
df_summary

,baseline_intercept,baseline_slope,mean,noise,std,area,height,sn,acquisition,experiment,precursor_mz,product_mz_start,product_mz_end,double_peak
0,acq,acq,acq,acq,acq,acq,acq,acq,acq,exp,premz,pro_mz_strat,promzend,
1,acq,acq,acq,acq,acq,acq,acq,acq,acq,exp,premz,pro_mz_strat,promzend,
2,acq,acq,acq,acq,acq,acq,acq,acq,acq,exp,premz,pro_mz_strat,promzend,
3,acq,acq,acq,acq,acq,acq,acq,acq,acq,exp,premz,pro_mz_strat,promzend,
4,acq,acq,acq,acq,acq,acq,acq,acq,acq,exp,premz,pro_mz_strat,promzend,
5,acq,acq,acq,acq,acq,acq,acq,acq,acq,exp,premz,pro_mz_strat,promzend,
6,acq,acq,acq,acq,acq,acq,acq,acq,acq,exp,premz,pro_mz_strat,promzend,
7,acq,acq,acq,acq,acq,acq,acq,acq,acq,exp,premz,pro_mz_strat,promzend,


In [20]:
df = pandas.DataFrame(
        {
            "baseline_intercept": {
                "mean": [np.nan],
                "sd": [np.nan],
                "hdi_3%": [np.nan],
                "hdi_97%": [np.nan],
                "mcse_mean": [np.nan],
                "mcse_sd": [np.nan],
                "ess_bulk": [np.nan],
                "ess_tail": [np.nan],
                "r_hat": [np.nan],
            },
            "baseline_slope": {
                "mean": [np.nan],
                "sd": [np.nan],
                "hdi_3%": [np.nan],
                "hdi_97%": [np.nan],
                "mcse_mean": [np.nan],
                "mcse_sd": [np.nan],
                "ess_bulk": [np.nan],
                "ess_tail": [np.nan],
                "r_hat": [np.nan],
            },
            "mean": {
                "mean": [np.nan],
                "sd": [np.nan],
                "hdi_3%": [np.nan],
                "hdi_97%": [np.nan],
                "mcse_mean": [np.nan],
                "mcse_sd": [np.nan],
                "ess_bulk": [np.nan],
                "ess_tail": [np.nan],
                "r_hat": [np.nan],
            },
            "noise": {
                "mean": [np.nan],
                "sd": [np.nan],
                "hdi_3%": [np.nan],
                "hdi_97%": [np.nan],
                "mcse_mean": [np.nan],
                "mcse_sd": [np.nan],
                "ess_bulk": [np.nan],
                "ess_tail": [np.nan],
                "r_hat": [np.nan],
            },
            "std": {
                "mean": [np.nan],
                "sd": [np.nan],
                "hdi_3%": [np.nan],
                "hdi_97%": [np.nan],
                "mcse_mean": [np.nan],
                "mcse_sd": [np.nan],
                "ess_bulk": [np.nan],
                "ess_tail": [np.nan],
                "r_hat": [np.nan],
            },
            "area": {
                "mean": [np.nan],
                "sd": [np.nan],
                "hdi_3%": [np.nan],
                "hdi_97%": [np.nan],
                "mcse_mean": [np.nan],
                "mcse_sd": [np.nan],
                "ess_bulk": [np.nan],
                "ess_tail": [np.nan],
                "r_hat": [np.nan],
            },
            "height": {
                "mean": [np.nan],
                "sd": [np.nan],
                "hdi_3%": [np.nan],
                "hdi_97%": [np.nan],
                "mcse_mean": [np.nan],
                "mcse_sd": [np.nan],
                "ess_bulk": [np.nan],
                "ess_tail": [np.nan],
                "r_hat": [np.nan],
            },
            "sn": {
                "mean": [np.nan],
                "sd": [np.nan],
                "hdi_3%": [np.nan],
                "hdi_97%": [np.nan],
                "mcse_mean": [np.nan],
                "mcse_sd": [np.nan],
                "ess_bulk": [np.nan],
                "ess_tail": [np.nan],
                "r_hat": [np.nan],
            },
        }
    )
# add information about the signal
df["acquisition"] = len(df.index) * [f"{ui.acquisition}"]
df["experiment"] = len(df.index) * [f"{ui.experiment}"]
df["precursor_mz"] = len(df.index) * [f"{ui.precursor_mz}"]
df["product_mz_start"] = len(df.index) * [f"{ui.product_mz_start}"]
df["product_mz_end"] = len(df.index) * [f"{ui.product_mz_end}"]
df["double_peak"] = len(df.index) * [False]
# concatenate to existing summary DataFrame
df

,baseline_intercept,baseline_slope,mean,noise,std,area,height,sn,acquisition,experiment,precursor_mz,product_mz_start,product_mz_end,double_peak
mean,[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],A1t1R1Part2,3,111.0,110.9,111.1,False
sd,[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],A1t1R1Part2,3,111.0,110.9,111.1,False
hdi_3%,[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],A1t1R1Part2,3,111.0,110.9,111.1,False
hdi_97%,[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],A1t1R1Part2,3,111.0,110.9,111.1,False
mcse_mean,[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],A1t1R1Part2,3,111.0,110.9,111.1,False
mcse_sd,[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],A1t1R1Part2,3,111.0,110.9,111.1,False
ess_bulk,[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],A1t1R1Part2,3,111.0,110.9,111.1,False
ess_tail,[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],A1t1R1Part2,3,111.0,110.9,111.1,False
r_hat,[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],A1t1R1Part2,3,111.0,110.9,111.1,False


In [19]:
df_summary = pandas.concat([df_summary, df])
df_summary

,baseline_intercept,baseline_slope,mean,noise,std,area,height,sn,acquisition,experiment,...,product_mz_end,double_peak,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
0,acq,acq,acq,acq,acq,acq,acq,acq,acq,exp,...,promzend,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,acq,acq,acq,acq,acq,acq,acq,acq,acq,exp,...,promzend,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,acq,acq,acq,acq,acq,acq,acq,acq,acq,exp,...,promzend,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,acq,acq,acq,acq,acq,acq,acq,acq,acq,exp,...,promzend,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,acq,acq,acq,acq,acq,acq,acq,acq,acq,exp,...,promzend,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,acq,acq,acq,acq,acq,acq,acq,acq,acq,exp,...,promzend,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,acq,acq,acq,acq,acq,acq,acq,acq,acq,exp,...,promzend,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,acq,acq,acq,acq,acq,acq,acq,acq,acq,exp,...,promzend,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
baseline_intercept,NaN,NaN,[nan],NaN,NaN,NaN,NaN,NaN,A1t1R1Part2,3,...,111.1,False,[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan]
baseline_slope,NaN,NaN,[nan],NaN,NaN,NaN,NaN,NaN,A1t1R1Part2,3,...,111.1,False,[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan]


In [ ]:
# TODO: fix tests within user_info() method which somehow destroy the dictionary
ui.user_info

{'A1t1R1Part2_1_110_109.9_110.1.npy': (False, 26.2),
 'A1t1R1Part2_2_111_109.9_110.1.npy': (False, 26.2),
 'A1t1R1Part2_3_111_110.9_111.1.npy': (False, 26.2),
 'A1t1R1Part2_4_112_110.9_111.1.npy': (False, 26.2),
 'A1t1R1Part2_5_112_111.9_112.1.npy': (False, 26.2),
 'peak_width_estimate': 1,
 'pre_filtering': True}